In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import fasttext
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from nltk.tokenize import sent_tokenize
from pymorphy2 import MorphAnalyzer
import rouge

In [2]:
word_embeddings = {}
f = open('multilingual_embeddings.ru\multilingual_embeddings.ru', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word]= coefs
f.close()

In [3]:
fast_text_model = fasttext.load_model("ft_native_300_ru_wiki_lenta_nltk_word_tokenize.bin")

## Загружаем тексты

In [4]:
df = pd.read_csv("Tech.csv", sep="|")
df.head()

,Unnamed: 0,article_title,article_text,article_annotation
0,0,ИНТЕГРАЛЬНАЯ СИСТЕМА ВИНИТИ РАН ПО ИНФОРМАЦИОН...,ИНТЕГРАЛЬНАЯ СИСТЕМА ВИНИТИ РАН ПО ИНФОРМАЦИОН...,Рассмотрен опыт проектирования и основные напр...
1,1,Содержание структурных компонентов информацион...,"адъюнкт, Санкт-Петербургский военный институт ...","В статье рассматриваются компетенции, которые ..."
2,2,Геоинформационная система ГрафИн 4. 0 и ее при...,Описывается геоинформационная система ГрафИн 4...,Описывается геоинформационная система ГрафИн 4...
3,3,Медицинские информационные ресурсы сети Интернет,Кадырова Эльвира Алиевна — кандидат педагогиче...,В статье приводится обзор электронных информац...
4,4,Информационно-вычислительная среда «Электрофиз...,"﻿АВЕРЬЯНОВ1 Герман Петрович, кандидат техничес...","Рассматриваемая работа, которая проводится лаб..."


## Preprocessing

In [5]:
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()
def preprocess(doc, lemmatize, min_words=5):
    sentences = sent_tokenize(doc)
    doc_tokens = []
    filtered_sentences = []
    sentence_indexes = []
    for i, sentence in enumerate(sentences):
        tokens = []
        for token in sentence.split():
            if token not in stopwords_ru:
                token = re.sub("(\W+|\d|[A-Za-z_])", '', token)
                token = token.strip()
                if lemmatize:
                    token = morph.normal_forms(token)[0]
                if token:
                    tokens.append(token)
        if len(tokens) >= min_words:
            doc_tokens.append(tokens)
            sentence_indexes.append(i)
            filtered_sentences.append(sentence)
    return (doc_tokens, filtered_sentences)

Glove

In [6]:
def glove_vectors(word):
    return word_embeddings.get(word, np.zeros((300,)))

FastText

In [7]:
def fast_text_vectors(word):
    return fast_text_model[word]


TextRank

In [8]:
def text_rank(sentences, vector_provider, dimension=300):
    sentence_vectors = []
    for sentence in sentences:
        vector = sum([vector_provider(word) for word in sentence])/(len(sentence)+0.001)
        sentence_vectors.append(vector)
    
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,dimension), sentence_vectors[j].reshape(1,dimension))[0,0]
    
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank_numpy(nx_graph)
    ranks = sorted(((scores[i],i) for i in range(len(sentences))), reverse=True)
    return ranks

Summarize

In [9]:
def summarize(doc, vectorizer, lemmatize, summary_count=3):
    words, sentences = preprocess(doc, lemmatize)
    ranked = text_rank(words, vectorizer)
    top = ranked[:summary_count]
    return [sentences[idx] for _,idx in top]

In [11]:
print("Название статьи:\n", df['article_title'][281])
print("Оригинальная аннотация:\n", df['article_annotation'][281])
print("GloveLemmatize:\n", ".\n\n".join(summarize(df['article_text'][281], glove_vectors, True)))
print("Glove:\n", ".\n\n".join(summarize(df['article_text'][281], glove_vectors, True)))
print("FastTextLemmatize:\n", ".\n\n".join(summarize(df['article_text'][281], fast_text_vectors, True)))
print("FastText:\n", ".\n\n".join(summarize(df['article_text'][281], fast_text_vectors, False)))

Название статьи:
 Автоматизация технологических процессов предоставления литературы пользователям БЕН РАН
Оригинальная аннотация:
 Описано функционирование разработанных и внедренных в БЕН РАН Интернет-систем заказа литера-туры по МБА и в читальном зале.Functioning of the developed and implemented Internet-based systems for ordering literature on interlibrary loan and in a reading room in the Library on natural sciences is described.
GloveLemmatize:
 Основные функции системы следующие:
•	предоставление абонентам возможности заказа материалов через Интернет с использованием сводных каталогов ЦБС БЕН РАН;
•	диспетчеризация заказов, полученных по традиционной почте и через Интернет (регистрация поступлений заказов; шифровка; регистрация выполнения заказов, отказов, возврата изданий; перенаправление заказов в другие библиотеки);
В состав комплексной автоматизированной системы МБА БЕН РАН входит Интернет-система формирования заказов, позволяющая зарегистрированным абонентам с сайта БЕН РАН 

Compare with reference

In [11]:
rouger = rouge.Rouge()

def summarize_and_compare(text, annotation, lemmatize, vectorizer):
    annotation_len = len(sent_tokenize(annotation))
    summary = summarize(text, vectorizer, lemmatize, annotation_len)
    scores = rouger.get_scores(str(summary), str(annotation))
    only_f_scores = {x:y['f'] for x,y in scores[0].items()}
    only_f_scores['summary'] = "\n".join(summary)
    return only_f_scores

def meter_one_doc(text, annotation):
    ft_res = summarize_and_compare(text, annotation, False, fast_text_vectors)
    gv_res = summarize_and_compare(text, annotation, False, glove_vectors)
    ft_res_l = summarize_and_compare(text, annotation, True, fast_text_vectors)
    gv_res_l = summarize_and_compare(text, annotation, True, glove_vectors)
    result = {'text':text, 
              'annotation':annotation, 
              'gv_summary':gv_res['summary'],
              'gv_rouge_1':gv_res['rouge-1'],
              'gv_rouge_2':gv_res['rouge-2'],
              'gv_rouge_l':gv_res['rouge-l'],
              'gvl_summary':gv_res_l['summary'],
              'gvl_rouge_1':gv_res_l['rouge-1'],
              'gvl_rouge_2':gv_res_l['rouge-2'],
              'gvl_rouge_l':gv_res_l['rouge-l'],
              'ft_summary':ft_res['summary'],
              'ft_rouge_1':ft_res['rouge-1'],
              'ft_rouge_2':ft_res['rouge-2'],
              'ft_rouge_l':ft_res['rouge-l'],
              'ftl_summary':ft_res_l['summary'],
              'ftl_rouge_1':ft_res_l['rouge-1'],
              'ftl_rouge_2':ft_res_l['rouge-2'],
              'ftl_rouge_l':ft_res_l['rouge-l']
             }
    return result

In [12]:
resultdf = None
for i, title, text, annotation in df.itertuples(index=False):
    row = meter_one_doc(text, annotation)
    row['title'] = title
    if resultdf is not None:
        resultdf = resultdf.append(pd.DataFrame.from_records([row]), ignore_index=True)
    else:
        resultdf = pd.DataFrame.from_records([row])
    print("done "+str(i)+"th text")
resultdf

done 0th text
done 1th text
done 2th text
done 3th text
done 4th text
done 5th text
done 6th text
done 7th text
done 8th text
done 9th text
done 10th text
done 11th text
done 12th text
done 13th text
done 14th text
done 15th text
done 16th text
done 17th text
done 18th text
done 19th text
done 20th text
done 21th text
done 22th text
done 23th text
done 24th text
done 25th text
done 26th text
done 27th text
done 28th text
done 29th text
done 30th text
done 31th text
done 32th text
done 33th text
done 34th text
done 35th text
done 36th text
done 37th text
done 38th text
done 39th text
done 40th text
done 41th text
done 42th text
done 43th text
done 44th text
done 45th text
done 46th text
done 47th text
done 48th text
done 49th text
done 50th text
done 51th text
done 52th text
done 53th text
done 54th text
done 55th text
done 56th text
done 57th text
done 58th text
done 59th text
done 60th text
done 61th text
done 62th text
done 63th text
done 64th text
done 65th text
done 66th text
done 

,text,annotation,gv_summary,gv_rouge_1,gv_rouge_2,gv_rouge_l,gvl_summary,gvl_rouge_1,gvl_rouge_2,gvl_rouge_l,ft_summary,ft_rouge_1,ft_rouge_2,ft_rouge_l,ftl_summary,ftl_rouge_1,ftl_rouge_2,ftl_rouge_l,title
0,ИНТЕГРАЛЬНАЯ СИСТЕМА ВИНИТИ РАН ПО ИНФОРМАЦИОН...,Рассмотрен опыт проектирования и основные напр...,В результате анализа информационных ресурсов в...,0.173160,0.008734,0.088889,В результате анализа информационных ресурсов в...,0.153846,0.008163,0.081218,"Совершенно очевидно, что формирование ИИС ВИНИ...",0.159664,0.016949,0.107527,Для решения поставленной задачи необходимо:\n•...,0.181070,0.016598,0.115183,ИНТЕГРАЛЬНАЯ СИСТЕМА ВИНИТИ РАН ПО ИНФОРМАЦИОН...
1,"адъюнкт, Санкт-Петербургский военный институт ...","В статье рассматриваются компетенции, которые ...","•\tво-первых, с постепенным развитием и увелич...",0.180082,0.032832,0.147410,"•\tво-первых, с постепенным развитием и увелич...",0.178470,0.019886,0.136546,"•\tво-первых, с постепенным развитием и увелич...",0.166898,0.016736,0.114851,«Способность самосовершенствоваться и повышать...,0.167131,0.016760,0.125000,Содержание структурных компонентов информацион...
2,Описывается геоинформационная система ГрафИн 4...,Описывается геоинформационная система ГрафИн 4...,"Например, опытный пользователь может создать с...",0.000000,0.000000,0.000000,Для условных знаков фиксированного масштаба об...,0.024691,0.000000,0.025316,САПРовский слой векторной графики предназначен...,0.037037,0.000000,0.038462,В последней версии геоинформационной системы Г...,0.092593,0.018868,0.111111,Геоинформационная система ГрафИн 4. 0 и ее при...
3,Кадырова Эльвира Алиевна — кандидат педагогиче...,В статье приводится обзор электронных информац...,Значительный вклад в формирование информационн...,0.115830,0.023346,0.075829,На базе фондов ЦНМБ создается Федеральная элек...,0.130081,0.049180,0.108911,"В числе библиографических источников, ориентир...",0.129032,0.037209,0.109290,и является крупнейшей реферативной базой данны...,0.092437,0.033898,0.068966,Медицинские информационные ресурсы сети Интернет
4,"﻿АВЕРЬЯНОВ1 Герман Петрович, кандидат техничес...","Рассматриваемая работа, которая проводится лаб...",Второй — функциональный или технологический ас...,0.027027,0.000000,0.015504,"И хотя информация в студенческой базе данных, ...",0.146341,0.050000,0.103896,);\n♦\tсложность полномасштабного натурного (ф...,0.076923,0.000000,0.035714,);\n♦\tсложность полномасштабного натурного (ф...,0.076923,0.000000,0.035714,Информационно-вычислительная среда «Электрофиз...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,В лекции излагается современный взгляд на неко...,В лекции излагается современный взгляд на неко...,"Кроме того, в последнее время активно учитываю...",0.102190,0.000000,0.093023,"Кроме того, в последнее время активно учитываю...",0.101695,0.000000,0.103226,"Кроме того, в последнее время активно учитываю...",0.081395,0.000000,0.075000,"Кроме того, в последнее время активно учитываю...",0.101695,0.000000,0.103226,Лингвистические средства информационного поиск...
278,Предлагается «квантовый» процессор для моделир...,Предлагается «квантовый» процессор для моделир...,Практическая значимость нового подхода синтеза...,0.259819,0.036474,0.132841,Практическая значимость нового подхода синтеза...,0.443730,0.291262,0.385542,Предлагается модификация упомянутой модели авт...,0.410405,0.255814,0.349442,Используются квантовые или кубитные структуры ...,0.440367,0.270769,0.366412,Синтез и анализ «Квантовых» моделей цифровых с...
279,"﻿обучения, учебной и производственной практики...",В статье приводится обоснование необходимости ...,"вокупности представлять собой систему, основан...",0.172840,0.025316,0.138889,"вокупности представлять собой систему, основан...",0.172840,0.025316,0.138889,В соответствии с принципами «процессного» и «с...,0.084746,0.000000,0.037383,В соответствии с принципами «процессного» и «с...,0.084746,0.000000,0.037383,ИНФОРМАЦИОННОЕ ОБЕСПЕЧЕНИЕ СИСТЕМЫ МЕНЕДЖМ

In [13]:
resultdf.to_csv('results.csv', sep="|")


In [3]:
dfres = pd.read_csv("results.csv", sep="|")

In [29]:
xres = dfres[['text', 'annotation', 'gv_summary', 'gvl_summary', 'ft_summary', 'ftl_summary']]
xres
# xres.to_csv("results_summ.csv", sep="|", index=False)

,text,annotation,gv_summary,gvl_summary,ft_summary,ftl_summary


In [24]:
#выбросы
dfres = dfres[np.abs(dfres['gv_rouge_1']-dfres['gv_rouge_1'].mean())<=(3*dfres['gv_rouge_1'].std())]
dfres = dfres[np.abs(dfres['gv_rouge_2']-dfres['gv_rouge_2'].mean())<=(3*dfres['gv_rouge_2'].std())]
dfres = dfres[np.abs(dfres['gv_rouge_l']-dfres['gv_rouge_l'].mean())<=(3*dfres['gv_rouge_l'].std())]
dfres = dfres[np.abs(dfres['ft_rouge_1']-dfres['ft_rouge_1'].mean())<=(3*dfres['ft_rouge_1'].std())]
dfres = dfres[np.abs(dfres['ft_rouge_2']-dfres['ft_rouge_2'].mean())<=(3*dfres['ft_rouge_2'].std())]
dfres = dfres[np.abs(dfres['ft_rouge_l']-dfres['ft_rouge_l'].mean())<=(3*dfres['ft_rouge_l'].std())]

In [28]:
print('gv_rouge_1', dfres['gv_rouge_1'].mean())
print('gv_rouge_2', dfres['gv_rouge_2'].mean())
print('gv_rouge_l', dfres['gv_rouge_l'].mean())
print('ft_rouge_1', dfres['ft_rouge_1'].mean())
print('ft_rouge_2', dfres['ft_rouge_2'].mean())
print('ft_rouge_l', dfres['ft_rouge_l'].mean())
print('gvl_rouge_1', dfres['gvl_rouge_1'].mean())
print('gvl_rouge_2', dfres['gvl_rouge_2'].mean())
print('gvl_rouge_l', dfres['gvl_rouge_l'].mean())
print('ftl_rouge_1', dfres['ftl_rouge_1'].mean())
print('ftl_rouge_2', dfres['ftl_rouge_2'].mean())
print('ftl_rouge_l', dfres['ftl_rouge_l'].mean())

gv_rouge_1 0.17230916869986743
gv_rouge_2 0.053631913015659206
gv_rouge_l 0.14416788686233184
ft_rouge_1 0.17271296631121602
ft_rouge_2 0.05280365841852497
ft_rouge_l 0.14404203533579066
gvl_rouge_1 0.1801783659363448
gvl_rouge_2 0.06173532220059998
gvl_rouge_l 0.15196920794026827
ftl_rouge_1 0.18049398566619962
ftl_rouge_2 0.060457915656514725
ftl_rouge_l 0.15144385406761968


In [30]:
import scipy.stats as stats

In [32]:
print('gv_1: ', stats.ttest_rel(dfres['gv_rouge_1'], dfres['gvl_rouge_1']))
print('gv_2: ', stats.ttest_rel(dfres['gv_rouge_2'], dfres['gvl_rouge_2']))
print('gv_l: ', stats.ttest_rel(dfres['gv_rouge_l'], dfres['gvl_rouge_l']))#22
print('ft_1: ', stats.ttest_rel(dfres['ft_rouge_1'], dfres['ftl_rouge_1']))
print('ft_2: ', stats.ttest_rel(dfres['ft_rouge_2'], dfres['ftl_rouge_2']))
print('ft_l: ', stats.ttest_rel(dfres['ft_rouge_l'], dfres['ftl_rouge_l'])) #23

gv_1:  Ttest_relResult(statistic=-1.6509935510871845, pvalue=0.09992892908534254)
gv_2:  Ttest_relResult(statistic=-1.6353286400197582, pvalue=0.10317187451818494)
gv_l:  Ttest_relResult(statistic=-1.4991681931080236, pvalue=0.13502483834040133)
ft_1:  Ttest_relResult(statistic=-2.9253907665402936, pvalue=0.0037393603028915592)
ft_2:  Ttest_relResult(statistic=-2.895688094441982, pvalue=0.004099913619975429)
ft_l:  Ttest_relResult(statistic=-2.664586242697877, pvalue=0.008182925362796549)


In [33]:
print('gv_ft_1: ', stats.ttest_rel(dfres['gv_rouge_1'], dfres['ft_rouge_1']))
print('gv_ft_2: ', stats.ttest_rel(dfres['gv_rouge_2'], dfres['ft_rouge_2']))
print('gv_ft_l: ', stats.ttest_rel(dfres['gv_rouge_l'], dfres['ft_rouge_l'])) #24

gv_ft_1:  Ttest_relResult(statistic=-0.12431106886932462, pvalue=0.9011635814723011)
gv_ft_2:  Ttest_relResult(statistic=0.25534041084560855, pvalue=0.7986590069432835)
gv_ft_l:  Ttest_relResult(statistic=0.03534934440088803, pvalue=0.9718278896929338)


In [34]:
print('gvl_ftl_1: ', stats.ttest_rel(dfres['gvl_rouge_1'], dfres['ftl_rouge_1']))
print('gvl_ftl_2: ', stats.ttest_rel(dfres['gvl_rouge_2'], dfres['ftl_rouge_2']))
print('gvl_ftl_l: ', stats.ttest_rel(dfres['gvl_rouge_l'], dfres['ftl_rouge_l']))

gvl_ftl_1:  Ttest_relResult(statistic=-0.07117650712397804, pvalue=0.9433111675690313)
gvl_ftl_2:  Ttest_relResult(statistic=0.26372683425043886, pvalue=0.7921961532638236)
gvl_ftl_l:  Ttest_relResult(statistic=0.10493872786654249, pvalue=0.9165041117954582)


In [14]:
import csv

In [43]:
counter = 1
succ = 0
for row in xres.itertuples():
    counter += 1
    gv = sent_tokenize(row.gv_summary)
    gvl = sent_tokenize(row.gvl_summary)
    if set(gv) != set(gvl):
        succ += 1
        if succ < 10:
            print("INDEX:",row.Index,"\n")
            print('GLOVE:\n', "\n\n".join(gv))
            print('GLOVE_LEMM:\n', "\n\n".join(gvl))

INDEX: 0 

GLOVE:
 В результате анализа информационных ресурсов в области математических и физических наук и средств доступа к ним стала очевидной необходимость формирования в составе ИИС ВИНИТИ принципиально новых фрагментов этой системы.

Для эффективной работы системы информационного обеспечения научных исследований в области математических и физических наук БД «Математика» и «Физика» должны обладать следующими характеристиками:
•	расширяемость, т.е.

Для решения поставленной задачи необходимо:
•	расширить зону поиска релевантной информации и существенно снизить уровень «информационного шума»;
•	разработать интегральные классификационные системы (рубрикаторы по математическим и физическим наукам), позволяющие вести сквозной поиск необходимой информации в отечественных и зарубежных источниках;
•	разработать и обеспечить дальнейшее развитие системы лингвистического оснащения, обеспечивающей поиск релевантной информации;
•	обеспечить взаимодействие электронных информационных ресурсов (